In [18]:
%pip install --upgrade --quiet  gpt4all
%pip install langchain-community langchain langchain-chroma

Note: you may need to restart the kernel to use updated packages.

  Using cached build-1.2.2-py3-none-any.whl.metadata (6.2 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached onnxruntime-1.19.2-cp312-cp312-win_amd64.whl.metadata (4.7 kB)
  Using cached opentelemetry_api-1.27.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.27.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.48b0-py3-none-any.whl.metadata (2.1 kB)
  Using cached opentelemetry_sdk-1.27.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached PyPika-0.48.9-py2.py3-none-any.whl
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Usin

In [8]:
from langchain_community.embeddings import GPT4AllEmbeddings

 # GPT4All 임베딩 객체를 생성합니다.
gpt4all_embd = GPT4AllEmbeddings() 

Downloading: 100%|██████████| 45.9M/45.9M [00:02<00:00, 20.0MiB/s]
Verifying: 100%|██████████| 45.9M/45.9M [00:00<00:00, 582MiB/s]


In [9]:
text = (
    "임베딩 테스트를 하기 위한 샘플 문장입니다."  # 테스트용 문서 텍스트를 정의합니다.
)

In [10]:
query_result = gpt4all_embd.embed_query(
    text
)  # 주어진 텍스트에 대한 쿼리 임베딩을 생성합니다.

In [13]:
# 주어진 텍스트를 임베딩하여 문서 벡터를 생성합니다.
doc_result = gpt4all_embd.embed_documents([text])

In [23]:
# 임베딩된 차원의 크기를 확인합니다.
len(doc_result[0])

384

### Chroma 벡터스토어

In [28]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [29]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# %pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("VectorStores")

LangSmith 추적을 시작합니다.
[프로젝트명]
VectorStores


In [31]:
from langchain_community.document_loaders import TextLoader
# from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma


# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)

# 텍스트 파일을 load -> List[Document] 형태로 변환
loader1 = TextLoader("news.txt", encoding='utf-8')
loader2 = TextLoader("news2.txt", encoding='utf-8')

# 문서 분할
split_doc1 = loader1.load_and_split(text_splitter)
split_doc2 = loader2.load_and_split(text_splitter)

# 문서 개수 확인
len(split_doc1), len(split_doc2)

RuntimeError: Error loading news.txt

### VectorStore 생성

In [ ]:
db = Chroma.from_documents(
    documents=split_doc1, embedding=OpenAIEmbeddings(), collection_name="my_db"
)

In [ ]:
DB_PATH = "./chroma_db"

# 문서를 디스크에 저장합니다. 저장시 persist_directory에 저장할 경로를 지정합니다.
persist_db = Chroma.from_documents(
    split_doc1, OpenAIEmbeddings(), persist_directory=DB_PATH, collection_name="my_db"
)

In [ ]:
persist_db = Chroma(
    persist_directory=DB_PATH,
    embedding_function=GPT4AllEmbeddings(),
    collection_name="my_db",
)

In [ ]:
persist_db.get()